<a href="https://colab.research.google.com/github/nayanesilv4/MentorIA/blob/main/MentorIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import google.generativeai as genai
from google.colab import userdata
import time

In [ ]:
SYSTEM_PROMPT = """
Você é MentorIA, um chatbot amigável, prestativo e encorajador, projetado para ajudar
adolescentes e adultos que estão ingressando na universidade pela primeira vez no Brasil.
Sua função é explicar termos acadêmicos de forma simples e clara e gerar orientações
iniciais personalizadas para quem está começando.

Características importantes:
- Use linguagem acessível, evite jargões complexos (a menos que esteja explicando um!).
- Seja paciente e compreensivo com as dúvidas de iniciantes.
- Transmita confiança e encorajamento para a nova fase da vida universitária.
- Mantenha o foco em adaptação, estudo, recursos universitários e dicas gerais para calouros.
- NÃO forneça informações oficiais sobre matrícula, prazos específicos da universidade, questões financeiras, ou conselhos que exijam conhecimento oficial das regras de uma instituição específica (a menos que seja um exemplo genérico e claro). Se a pergunta for muito específica sobre "como se matricular em X disciplina na universidade Y", diga que a informação oficial deve ser buscada na secretaria ou coordenação do curso.
- Responda de forma concisa, mas completa o suficiente para ser útil.
- Seja sucinto, responda de forma clara, objetiva e empática.

Sempre responda como o MentorIA, um guia experiente e acolhedor.
"""
try:
    API_KEY = userdata.get('GOOGLE_API_KEY')
    if not API_KEY:
         raise ValueError("Chave GOOGLE_API_KEY não encontrada nos Secrets do Colab.")

    genai.configure(api_key=API_KEY)

    model = genai.GenerativeModel(
        'gemini-2.0-flash',
         system_instruction=SYSTEM_PROMPT
    )

    test_response = model.generate_content("Olá, teste de configuração.")
    if not test_response or not test_response.text:
         raise Exception("Modelo Gemini não respondeu corretamente ao teste.")
    ia_disponivel = True
    print("IA configurada e pronta para usar!")

except Exception as e:
    print(f"ERRO: Problema ao configurar a IA. Detalhes: {e}")
    print("As funcionalidades que usam IA (Orientação e Termos) não estarão disponíveis.")
    ia_disponivel = False
    model = None

IA configurada e pronta para usar!


In [ ]:
def pedir_para_ia(prompt):
    if not ia_disponivel or not model:
        return "Desculpe, a funcionalidade de IA não está ativa no momento."

    try:
        response = model.generate_content(prompt)
        if response and hasattr(response, 'text') and response.text:
             return response.text
        elif response and response.prompt_feedback:
            if response.prompt_feedback.block_reason:
                 return "Desculpe, sua solicitação não pôde ser processada. Ela violou alguma política de conteúdo."
            else:
                 return "Desculpe, não consegui gerar uma resposta para isso."
        else:
             return "Desculpe, não consegui gerar uma resposta para isso."

    except Exception as e:
        print(f"Erro ao chamar a IA: {e}")
        return "Desculpe, ocorreu um erro ao processar sua solicitação com a IA."

In [ ]:
def mostrar_menu_principal():
    print("\n" + "=" * 40)
    print("Bem-vindo(a) ao MentorIA!")
    print("Sou seu assistente virtual para ajudar na sua jornada universitária.")
    if not ia_disponivel:
         print("\nAVISO: A IA não está funcionando. Algumas opções podem não estar disponíveis.")
    print("=" * 40)

    print("Escolha uma opção digitando o número correspondente:")
    print("1. Gerar Orientação Inicial")
    print("2. Explicar Termo Acadêmico")
    print("3. Sair")
    print("-" * 40)

In [ ]:
def modo_orientacao():
    print("\n" + "*" * 40)
    print("  MODO: GERAR ORIENTAÇÃO INICIAL")
    print("*" * 40)

    if not ia_disponivel:
         print("Este modo requer a funcionalidade de IA, que não está disponível.")
         time.sleep(2)
         return

    print("Vamos conversar um pouco para o MentorIA montar sua orientação inicial.")
    print("Digite 'menu' a qualquer momento para voltar ao Menu Principal.")

    curso = input("Qual curso você vai começar? ").strip()
    expectativa = input("Qual sua maior expectativa ou preocupação ao iniciar a faculdade? ").strip()
    conhece_campus = input("Você já conhece o campus ou sabe onde encontrar informações (biblioteca, RU, coordenação)? (sim/não) ").lower().strip()

    print("\n" + "-" * 40)
    print("MentorIA está pensando para gerar sua Orientação Inicial...")
    time.sleep(2)

    orientation_prompt_initial = f"""
    Gere uma orientação inicial para um estudante universitário de primeira viagem no Brasil, focando em:
    - Boas-vindas e adaptação à vida universitária brasileira.
    - Dicas relevantes para o curso: {curso if curso else 'não especificado'}.
    - Como lidar com a preocupação/expectativa mencionada: {expectativa if expectativa else 'não especificado'}.
    - Dicas sobre explorar o campus/recursos, considerando que ele(a) {'não ' if conhece_campus != 'sim' else ''}conhece bem o campus.
    - Encorajamento geral e dicas para o sucesso nos primeiros semestres.

    Formate a resposta com tópicos claros e linguagem acessível, no tom amigável do MentorIA. Seja o mais sucinto possivel
    """

    orientacao_texto = pedir_para_ia(orientation_prompt_initial)
    print("\n" + "-" * 40)
    print(orientacao_texto)
    print("-" * 40)

    while True:
        print("\n" + "*" * 40)
        proxima_interacao = input("Mais alguma dúvida sobre a vida universitária? (Digite sua pergunta) Ou quer voltar? (Digite 'menu'): ").strip()

        if proxima_interacao.lower() == 'menu':
            print("Saindo do modo Orientação...")
            time.sleep(1)
            break
        elif proxima_interacao:
            print("\nMentorIA está pensando na sua dúvida...")
            time.sleep(1)
            follow_up_prompt = f"""
            O usuário, um estudante universitário de primeira viagem, está no modo de orientação e tem a seguinte dúvida adicional sobre a vida universitária:
            Pergunta: "{proxima_interacao}"
            Por favor, responda a esta dúvida mantendo o contexto de orientação para calouros e o seu tom de MentorIA. Seja direto e útil.
            """
            resposta_follow_up = pedir_para_ia(follow_up_prompt)
            print("\n" + "-" * 40)
            print(resposta_follow_up)
            print("-" * 40)
        else:
            print("Por favor, digite sua pergunta ou 'menu' para sair.")

In [ ]:
def modo_explicar_termo():
    print("\n" + "*" * 40)
    print("  MODO: EXPLICAR TERMO ACADÊMICO")
    print("*" * 40)

    if not ia_disponivel:
         print("Este modo requer a funcionalidade de IA, que não está disponível.")
         time.sleep(2)
         return

    print("Digite o termo acadêmico que você gostaria de entender.")
    print("Digite 'menu' a qualquer momento para voltar ao Menu Principal.")

    while True:
        termo_usuario = input("\nDigite o termo (ou 'menu' para sair): ").strip()

        if termo_usuario.lower() == 'menu':
            print("Saindo do modo Explicar Termo...")
            time.sleep(1)
            break

        elif termo_usuario:
            print(f"\nMentorIA está pensando na explicação para '{termo_usuario}'...")
            time.sleep(1)
            term_prompt = f"Explique o termo acadêmico '{termo_usuario}' para um estudante que está começando a universidade no Brasil. Use linguagem muito simples e direta, no tom amigável do MentorIA. Seja o mais sucinto possível"

            explicacao = pedir_para_ia(term_prompt)
            print("\n" + "-" * 40)
            print(f"Explicação para '{termo_usuario}':")
            print(explicacao)
            print("-" * 40)
        else:
             print("Por favor, digite um termo para explicar ou 'menu' para sair.")

In [ ]:
if __name__ == "__main__":
    while True:
        mostrar_menu_principal()

        escolha = input("Sua escolha: ").strip()

        if escolha == '1':
            modo_orientacao()

        elif escolha == '2':
            modo_explicar_termo()

        elif escolha == '3':

            print("\n" + "=" * 40)
            print("Obrigado por usar o MentorIA!")
            print("Lembre-se: A jornada universitária é desafiadora, mas incrível.")
            print("Não tenha medo de buscar conhecimento e apoio. Você é capaz!")
            print("Até a próxima!")
            print("=" * 40)
            break

        else:
            print("\nOpção inválida. Por favor, digite 1, 2 ou 3.")
            time.sleep(1)